In [23]:
from glob import iglob
import os

import pandas as pd
import screed
import seaborn as sns
from tqdm import tqdm

# Change to Quest for Orthologs 2019 data directory

In [4]:
cd ~/data_sm/kmer-hashing/quest-for-orthologs/data/2019/

/mnt/data_sm/olga/kmer-hashing/quest-for-orthologs/data/2019


In [5]:
ls -lha

total 2.6G
drwxr-xr-x 5 olga root 4.0K Jan  8 16:08 ./
drwxr-xr-x 3 olga root 4.0K Dec 25 17:48 ../
drwxr-xr-x 5 olga czb  4.0K Dec 26 19:44 Archaea/
drwxr-xr-x 5 olga czb   16K Dec 26 19:44 Bacteria/
drwxr-xr-x 8 olga czb   32K Jan  8 08:13 Eukaryota/
-rw-r--r-- 1 olga czb   64M Jan  8 16:08 opisthokont_not_human_transcription_factors_ensembl_compara.csv
-rw-r--r-- 1 olga czb  2.6G Dec 25 18:46 QfO_release_2019_04.tar.gz
-rw-r--r-- 1 olga czb   18K May 10  2019 README
-rw-r--r-- 1 olga czb   12K Jan  8 07:47 species_metadata.csv


# Download orthology and transcription factor data

## Read orthologous transcription factors 

In [8]:
tfs = pd.read_csv('opisthokont_not_human_transcription_factors_ensembl_compara.csv')
print(tfs.shape)
tfs.head()

(343153, 16)


,dn_ds,method_link_type,source__id,source__perc_id,source__perc_pos,source__protein_id,source__species,source__taxon_id,target__id,target__perc_id,target__perc_pos,target__protein_id,target__species,target__taxon_id,taxonomy_level,type
0,NaN,ENSEMBL_PARALOGUES,ENSG00000267179,24.8792,37.1981,ENSP00000467286,homo_sapiens,9606,ENSG00000175787,17.0813,25.5390,ENSP00000378792,homo_sapiens,9606,Bilateria,other_paralog
1,NaN,ENSEMBL_PARALOGUES,ENSG00000267179,25.6039,37.1981,ENSP00000467286,homo_sapiens,9606,ENSG00000187815,20.1521,29.2776,ENSP00000361791,homo_sapiens,9606,Bilateria,other_paralog
2,0.35552,ENSEMBL_PARALOGUES,ENSG00000267179,26.3285,38.1642,ENSP00000467286,homo_sapiens,9606,ENSG00000172006,20.2602,29.3680,ENSP00000321132,homo_sapiens,9606,Bilateria,other_paralog
3,0.33996,ENSEMBL_PARALOGUES,ENSG00000267179,27.2947,38.8889,ENSP00000467286,homo_sapiens,9606,ENSG00000188629,23.4927,33.4719,ENSP00000415070,homo_sapiens,9606,Bilateria,other_paralog
4,0.18553,ENSEMBL_PARALOGUES,ENSG00000267179,20.2899,25.8454,ENSP00000467286,homo_sapiens,9606,ENSG00000270011,26.1682,33.3333,ENSP00000445323,homo_sapiens,9606,Bilateria,other_paralog


# Go to Quest for Orthologs fastas

## Read species metadata

In [9]:
species_metadata = pd.read_csv("species_metadata.csv")
print(species_metadata.shape)
species_metadata.head()

(78, 10)


,proteome_id,tax_id,oscode,n_main_fasta_canonical,n_additional_fasta_isoforms,n_gene2acc,species_name,divergence_from_human_mya,scientific_name,common_name_or_strain
0,UP000007062,7165,ANOGA,12553,971,13619,Anopheles gambiae (African malaria mosquito),797.0,Anopheles gambiae,African malaria mosquito
1,UP000000798,224324,AQUAE,1553,0,1557,Aquifex aeolicus (strain VF5),4290.0,Aquifex aeolicus,strain VF5
2,UP000006548,3702,ARATH,27475,14123,41920,Arabidopsis thaliana (Mouse-ear cress),1496.0,Arabidopsis thaliana,Mouse-ear cress
3,UP000001570,224308,BACSU,4260,7,4268,Bacillus subtilis (strain 168),4290.0,Bacillus subtilis,strain 168
4,UP000001414,226186,BACTN,4782,0,4823,Bacteroides thetaiotaomicron (strain ATCC 2914...,4290.0,Bacteroides thetaiotaomicron,strain ATCC 29148 / DSM 2079 / NCTC 10582 / E5...


### Subset to opisthokonts

In [10]:
# Estimated opisthokonta divergence time from http://timetree.org/
t = 1105
opisthokonts = species_metadata.query('divergence_from_human_mya <= @t')
print(opisthokonts.shape)
opisthokonts.head()

(35, 10)


,proteome_id,tax_id,oscode,n_main_fasta_canonical,n_additional_fasta_isoforms,n_gene2acc,species_name,divergence_from_human_mya,scientific_name,common_name_or_strain
0,UP000007062,7165,ANOGA,12553,971,13619,Anopheles gambiae (African malaria mosquito),797.0,Anopheles gambiae,African malaria mosquito
5,UP000007241,684364,BATDJ,8610,0,8685,Batrachochytrium dendrobatidis (strain JAM81 /...,1105.0,Batrachochytrium dendrobatidis,strain JAM81 / FGSC 10211
6,UP000009136,9913,BOVIN,23774,14534,38438,Bos taurus (Bovine),96.0,Bos taurus,Bovine
8,UP000001554,7739,BRAFL,28542,2,28608,Branchiostoma floridae (Florida lancelet) (Amp...,684.0,Branchiostoma floridae,Florida lancelet
9,UP000001940,6239,CAEEL,19986,8309,28507,Caenorhabditis elegans,797.0,Caenorhabditis elegans,NaN


In [11]:
opisthokonts.query('scientific_name == "Homo sapiens"')

,proteome_id,tax_id,oscode,n_main_fasta_canonical,n_additional_fasta_isoforms,n_gene2acc,species_name,divergence_from_human_mya,scientific_name,common_name_or_strain
32,UP000005640,9606,HUMAN,21146,74769,96332,Homo sapiens (Human),0.0,Homo sapiens,Human


## Read Gene Accession file

```
Gene mapping files (*.gene2acc)
===============================

Column 1 is a unique gene symbol that is chosen with the following order of
preference from the annotation found in:
1) Model Organism Database (MOD)
2) Ensembl or Ensembl Genomes database
3) UniProt Ordered Locus Name (OLN)
4) UniProt Open Reading Frame (ORF)
5) UniProt Gene Name
A dash symbol ('-') is used when the gene encoding a protein is unknown.

Column 2 is the UniProtKB accession or isoform identifier for the given gene
symbol. This column may have redundancy when two or more genes have identical
translations.

Column 3 is the gene symbol of the canonical accession used to represent the
respective gene group and the first row of the sequence is the canonical one.
```

In [109]:

def read_gene2acc(gene2acc, names=['maybe_ensembl_id', 'uniprot_id', 'canonical_accession']):
    df = pd.read_csv(gene2acc, sep='\t', header=None, na_values='-', names=columns)
    return df

gene2acc = read_gene2acc('Eukaryota/UP000005640_9606.gene2acc')
# gene2acc = pd.read_csv('Eukaryota/UP000005640_9606.gene2acc', sep='\t', header=None, na_values='-', names=columns)
print(gene2acc.shape)
gene2acc.head()

(96332, 3)


,maybe_ensembl_id,uniprot_id,canonical_accession
0,NaN,A0A075B7B6,NaN
1,NaN,A0A075B714,NaN
2,NaN,A0A075B713,NaN
3,NaN,A0A075B712,NaN
4,NaN,A0A075B711,NaN


In [29]:
gene2acc.dropna()

,maybe_ensembl_id,uniprot_id,canonical_accession
600,ACOT7L,Q6ZUV0,ACOT7L
601,BARGIN,Q6ZT62,BARGIN
602,BARGIN,Q6ZT62-2,BARGIN
603,BCE1,O60756,BCE1
604,C1orf140,Q5VVS0,C1orf140
...,...,...,...
96327,UNQ6190/PRO20217,Q6UXQ8,UNQ6190/PRO20217
96328,UNQ6493/PRO21345,Q6UXR8,UNQ6493/PRO21345
96329,UNQ6494/PRO21346,Q6UXR6,UNQ6494/PRO21346
96330,UNQ9165/PRO28630,Q6UXU0,UNQ9165/PRO28630


## Read ID mapping file

```
Database mapping files (*.idmapping)
====================================

These files contain mappings from UniProtKB to other databases for each
reference proteome.
The format consists of three tab-separated columns:

1. UniProtKB accession
2. ID_type:
   Database name as shown in UniProtKB cross-references and supported by the ID
   mapping tool on the UniProt web site (http://www.uniprot.org/mapping)
3. ID:
   Identifier in the cross-referenced database.

```

In [17]:
opisthokonts.head()

,proteome_id,tax_id,oscode,n_main_fasta_canonical,n_additional_fasta_isoforms,n_gene2acc,species_name,divergence_from_human_mya,scientific_name,common_name_or_strain
0,UP000007062,7165,ANOGA,12553,971,13619,Anopheles gambiae (African malaria mosquito),797.0,Anopheles gambiae,African malaria mosquito
5,UP000007241,684364,BATDJ,8610,0,8685,Batrachochytrium dendrobatidis (strain JAM81 /...,1105.0,Batrachochytrium dendrobatidis,strain JAM81 / FGSC 10211
6,UP000009136,9913,BOVIN,23774,14534,38438,Bos taurus (Bovine),96.0,Bos taurus,Bovine
8,UP000001554,7739,BRAFL,28542,2,28608,Branchiostoma floridae (Florida lancelet) (Amp...,684.0,Branchiostoma floridae,Florida lancelet
9,UP000001940,6239,CAEEL,19986,8309,28507,Caenorhabditis elegans,797.0,Caenorhabditis elegans,NaN


In [34]:
opisthokonts.query('proteome_id == "UP000000437"')

,proteome_id,tax_id,oscode,n_main_fasta_canonical,n_additional_fasta_isoforms,n_gene2acc,species_name,divergence_from_human_mya,scientific_name,common_name_or_strain
17,UP000000437,7955,DANRE,25939,21219,47517,Danio rerio (Zebrafish) (Brachydanio rerio),435.0,Danio rerio,Zebrafish


# Get ID Mapping for uniprot ids from ENSMBL

In [37]:
dfs = []

for filename in tqdm(sorted(iglob("Eukaryota/*.idmapping"))):
#     print(filename)
    basename = os.path.basename(filename)
    prefix = basename.split('.')[0]
    species_id, taxa_id = prefix.split("_")
#     print(f"{species_id=} {taxa_id=}")
    
    if species_id in opisthokonts.proteome_id.values:

        df = pd.read_csv(filename, sep='\t', header=None, names=['uniprot_id', 'id_type', 'db_id'])
        df['species_id'] = species_id
        df['taxa_id'] = species_id

        # Use only Ensembl data
#         df = df.query('id_type == "Ensembl"')
        print(df.shape)
        dfs.append(df)
    
id_mapping = pd.concat(dfs, ignore_index=True)
print(id_mapping.shape)
id_mapping.head()

  2%|▏         | 1/48 [00:00<00:32,  1.44it/s]

(1036083, 5)


 10%|█         | 5/48 [00:01<00:18,  2.32it/s]

(555421, 5)
(149092, 5)
(141961, 5)


 12%|█▎        | 6/48 [00:02<00:25,  1.62it/s]

...

(561122, 5)


100%|██████████| 48/48 [00:14<00:00,  3.24it/s]

(390075, 5)


(19464294, 5)


,uniprot_id,id_type,db_id,species_id,taxa_id
0,Q5PRD0,UniProtKB-ID,143BA_DANRE,UP000000437,UP000000437
1,Q5PRD0,Gene_Name,ywhaba,UP000000437,UP000000437
2,Q5PRD0,Gene_Synonym,ywhab1,UP000000437,UP000000437
3,Q5PRD0,Gene_ORFName,wu:fb80c08,UP000000437,UP000000437
4,Q5PRD0,GI,82592598,UP000000437,UP000000437


# Merge id mapping with ensembl compara tfs

In [40]:
tfs.tail()

,dn_ds,method_link_type,source__id,source__perc_id,source__perc_pos,source__protein_id,source__species,source__taxon_id,target__id,target__perc_id,target__perc_pos,target__protein_id,target__species,target__taxon_id,taxonomy_level,type
343148,NaN,ENSEMBL_ORTHOLOGUES,ENSG00000036549,36.43410,52.27020,ENSP00000359837,homo_sapiens,9606,ENSEBUG00000002277,39.2133,56.2575,ENSEBUP00000003097,eptatretus_burgeri,7764,Vertebrata,ortholog_one2one
343149,NaN,ENSEMBL_ORTHOLOGUES,ENSG00000036549,26.91030,33.44410,ENSP00000359837,homo_sapiens,9606,ENSPMAG00000000497,56.6434,70.3963,ENSPMAP00000000552,petromyzon_marinus,7757,Vertebrata,ortholog_one2one
343150,NaN,ENSEMBL_ORTHOLOGUES,ENSG00000036549,28.90370,45.29350,ENSP00000359837,homo_sapiens,9606,ENSCING00000002486,25.1203,39.3648,ENSCINP00000005056,ciona_intestinalis,7719,Chordata,ortholog_one2one
343151,NaN,ENSEMBL_ORTHOLOGUES,ENSG00000036549,7.08749,9.30233,ENSP00000359837,homo_sapiens,9606,ENSCSAVG00000000992,48.4848,63.6364,ENSCSAVP00000001713,ciona_savignyi,51511,Chordata,ortholog_one2one
343152,NaN,ENSEMBL_ORTHOLOGUES,ENSG00000036549,13.84270,21.04100,ENSP00000359837,homo_sapiens,9606,FBgn0031876,35.1124,53.3708,FBpp0078990,drosophila_melanogaster,7227,Bilateria,ortholog_one2one


In [41]:
tfs_uniprot = tfs.merge(id_mapping, left_on='target__id', right_on='db_id')
print(tfs_uniprot.shape)
tfs_uniprot.head()

(266814, 21)


,dn_ds,method_link_type,source__id,source__perc_id,source__perc_pos,source__protein_id,source__species,source__taxon_id,target__id,target__perc_id,...,target__protein_id,target__species,target__taxon_id,taxonomy_level,type,uniprot_id,id_type,db_id,species_id,taxa_id
0,NaN,ENSEMBL_PARALOGUES,ENSG00000267179,24.8792,37.1981,ENSP00000467286,homo_sapiens,9606,ENSG00000175787,17.0813,...,ENSP00000378792,homo_sapiens,9606,Bilateria,other_paralog,Q7Z761,Ensembl,ENSG00000175787,UP000005640,UP000005640
1,NaN,ENSEMBL_PARALOGUES,ENSG00000267179,24.8792,37.1981,ENSP00000467286,homo_sapiens,9606,ENSG00000175787,17.0813,...,ENSP00000378792,homo_sapiens,9606,Bilateria,other_paralog,Q14929,Ensembl,ENSG00000175787,UP000005640,UP000005640
2,NaN,ENSEMBL_PARALOGUES,ENSG00000267179,24.8792,37.1981,ENSP00000467286,homo_sapiens,9606,ENSG00000175787,17.0813,...,ENSP00000378792,homo_sapiens,9606,Bilateria,other_paralog,Q5SR55,Ensembl,ENSG00000175787,UP000005640,UP000005640
3,NaN,ENSEMBL_PARALOGUES,ENSG00000267179,24.8792,37.1981,ENSP00000467286,homo_sapiens,9606,ENSG00000175787,17.0813,...,ENSP00000378792,homo_sapiens,9606,Bilateria,other_paralog,S4R440,Ensembl,ENSG00000175787,UP000005640,UP000005640
4,0.3791,ENSEMBL_PARALOGUES,ENSG00000181666,45.6753,57.5114,ENSP00000315505,homo_sapiens,9606,ENSG00000175787,49.9171,...,ENSP00000378792,homo_sapiens,9606,Bilateria,other_paralog,Q7Z761,Ensembl,ENSG00000175787,UP000005640,UP000005640


In [42]:
tfs_uniprot.uniprot_id.nunique()

34146

## Make set variable for quick membership evalution

In [58]:
tf_orthologs = set(tfs_uniprot.uniprot_id)

In [59]:
%timeit 'Q7Z761' in tf_orthologs

45.3 ns ± 0.206 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [60]:
%timeit 'Q7Z761' in tfs_uniprot.uniprot_id

7.28 µs ± 135 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


# Read non-human proteins and subset if they are an ortholog of a TF

## Make outdir

In [43]:
not_human_outdir = 'Eukaryota/not-human-transcription-factor-fastas/'
! mkdir $not_human_outdir


mkdir: cannot create directory ‘Eukaryota/not-human-transcription-factor-fastas/’: File exists


## How much compute is this?

Number of human transcription factor proteins in the quest for orthologs database


In [55]:
n_human_qfo_tfs = 5749

In [50]:
tfs_uniprot.query('target__species != "homo_sapiens"').shape

(48947, 21)

In [53]:
n_not_human_qfo_tfs = 48947

In [57]:
n_human_qfo_tfs * n_not_human_qfo_tfs * 0.0006 / 60 / 60

46.89938383333333


### Okay, 46 hours is reasonable!

## Read in protein fastas with screed

In [61]:


for filename in iglob('Eukaryota/not-human-protein-fastas/*.fasta'):
    tf_records = []

    basename = os.path.basename(filename)
    with screed.open(filename) as records:
        for record in records:
            name = record['name']
            record_id = name.split()[0]
            uniprot_id = record_id.split('|')[1]
            if uniprot_id in tf_orthologs:
                tf_records.append(record)
                
    if len(tf_records) > 0:
        print(filename)
        print(f"\tlen(tf_records): {len(tf_records)}")
        with open(f'{not_human_outdir}/{basename}', 'w') as f:
            for record in tf_records:
                f.write(">{name}\n{sequence}\n".format(**record))


Eukaryota/not-human-protein-fastas/UP000000437_7955_additional.fasta
	len(tf_records): 1171
Eukaryota/not-human-protein-fastas/UP000008143_8364.fasta
	len(tf_records): 493
Eukaryota/not-human-protein-fastas/UP000001038_8090.fasta
	len(tf_records): 1263
Eukaryota/not-human-protein-fastas/UP000002280_13616.fasta
	len(tf_records): 1007
Eukaryota/not-human-protein-fastas/UP000000539_9031_additional.fasta
	len(tf_records): 640
Eukaryota/not-human-protein-fastas/UP000000803_7227_additional.fasta
	len(tf_records): 404
Eukaryota/not-human-protein-fastas/UP000018468_7918.fasta
	len(tf_records): 979
Eukaryota/not-human-protein-fastas/UP000002254_9615.fasta
	len(tf_records): 1231
Eukaryota/not-human-protein-fastas/UP000001940_6239.fasta
	len(tf_records): 544
Eukaryota/not-human-protein-fastas/UP000000437_7955.fasta
	len(tf_records): 1156
Eukaryota/not-human-protein-fastas/UP000002311_559292.fasta
	len(tf_records): 48
Eukaryota/not-human-protein-fastas/UP000000589_10090.fasta
	len(tf_records): 124

In [104]:
uniprot_id

'H6QVH5'